Final Assignment: 

 

Dr. Herrera has received tumor samples from numerous patients and is eager to explore the gene expression in a specific tumor sample. Rather than analyzing all patient samples, Dr. Herrera chose to randomly select one sample to gain preliminary insights into gene expression. His laboratory scientist processed the tumor tissues at a molecular level and dispatched a sample for NGS sequencing. The attached fastq files were returned to Dr. Herrera. Your assignment is to design an informatics workflow to analyze and report on the expressed genes in this sample. 

 

Initiate your bioinformatics approach by assessing the quality of the NGS fastq files. After the quality check, provide feedback on sequence quality and the number of reads present in the sample. You can employ built-in functions from Biopython for this step. Once the genes are identified, utilize a genome browser like IGV (Integrated Genome Viewer) at https://www.igv.org to visualize them. Hints: Apply the concepts learned in week 6 lectures and other relevant information to guide your analysis. (samtools, bedtools) 

 

Submit your script file along with a README. Additionally, discuss the relevance of your results in cancer biology. 

1. install biopython
2. import SeqIO - read and parse fastq file
3. find # of reads present in the sample
4. calculate quality scores
5. maybe visulize quality scores
6. Quality trimming if necessary? Trimmomatic?
7. download reference genome - either by using wget (bash) or by physically downloading
8. use bwq to index refrence genome
9. use star to align both paired reads to reference
11. samtools to convert to bam, sort, index
12. download refernce genome annotation file 
13. bedtools to intersect with annotation file (second intersect + save as txt file to extract gene names?)
14. extract unique gene names - use bash and pipes
15. ?? Quantify gene expression with bedtools
    - conert gtf annotation file to bed
    - count overlapping reads with bedtools coverage
16. visulize with IGV > load ref genome, then load sorted.bam file